### Load training dataset.

In [29]:
import os
import random
import pandas as pd

news = []

for directory in os.listdir("1150haber/raw_texts/"):
    for filename in os.listdir("1150haber/raw_texts/" + directory):
        f = open("1150haber/raw_texts/" + directory + "/" + filename, "r", encoding="windows-1254")
        new = f.read().replace("\n", " ")
        new = new.lower()
        news.append(new)
        
random.shuffle(news)
print(len(news))

data = {"Haber": news}
df_train = pd.DataFrame(data)
df_train["Haber"] = df_train["Haber"].apply(str)
df_train.sample(20)

100


,Haber
8,"başbakan recep tayyip erdoğan, irak'ın bütünlü..."
71,i̇stanbul üniversitesi (i̇ü) cerrahpaşa tıp fa...
89,zayıflamak isteyenlere hep kalori hesabı yapıl...
28,vücudunun ve yüzünün hiçbir yerine bıçak değdi...
63,'ruhban okulu' formülü hazır dışişleri bakanl...
0,arzu yeniden botaşspor’da bayanlar baske...
5,milli takım cnn’de lider ünlü haber kanalını...
50,rumlara ab şoku kıbrıs müzakerelerine ab...
82,washi̇ uyandirdi fenerbahçeli golcünün hasta...
4,taşlar yerine oturuyor ak parti hükümeti...


### Load test dataset.

In [4]:
import os
import random
import pandas as pd

news = []
labels = []
filenames = []
i = 0

for directory in os.listdir("1150haber/test/"):
    i += 1
    for filename in os.listdir("1150haber/test/" + directory):
        f = open("1150haber/test/" + directory + "/" + filename, "r", encoding="windows-1254")
        new = f.read().replace("\n", " ")
        new = new.lower()
        news.append(new)
        labels.append(i)
        
print(len(news))

data = {"Haber": news, "Sınıf": labels}
df_test = pd.DataFrame(data)
df_test["Haber"] = df_test["Haber"].apply(str)
df_test = df_test.sample(frac = 1).reset_index(drop=True)
df_test.sample(230).sort_index()

230


,Haber,Sınıf
0,efes turu kapti : 75-57 lacivert-beyazlılar ...,5
1,sakin pes etme fenerbahçe teknik direktö...,5
2,chp muhalefeti tatilde muhalifler ihraç edile...,4
3,"kartal’ın keyfi yerinde beşiktaş, fenerbahçe...",5
4,"gurbetçilerin tercihi ""o şarkılar"" albümü ya...",2
...,...,...
225,ssk’ya pahalı ilaç satarak devleti zarara uğra...,1
226,"doğum kayıtları, internette konya dr. faruk ...",3
227,abd: bizden iyi dostunuz yok ankara milliyet ...,4
228,"japan tobacco international (jti), geçen hafta...",1


In [5]:
EKONOMI = 1
MAGAZIN = 2
SAGLIK = 3
SIYASI = 4
SPOR = 5
BELIRSIZ = -1

In [13]:
ekonomi_kavramlar1 = ["lira", "ticaret", "merkez bankası", "ssk", "kredi", "maliye", "tüketim", "faiz"]
ekonomi_kavramlar2 = ["cari", "milyar", "milyon", "kurul", "tüketici", "dolar","ekonomi"]

magazin_kavramlar1 = ["manken", "konser", "eşi", "bodrum", "evlilik"]
magazin_kavramlar2 = ["sevgili", "güzellik", "nişantaşı", "mayo", "podyum"]

saglik_kavramlar1 = ["dr.", "sağlık", "koruyucu", "ilaç", "cilt", "kalsiyum", "fiziksel", "güneş", "hastalık", "hipertansiyon"]
saglik_kavramlar2 = ["ultraviyole", "alerji", "yağ", "estetik", "tedavi", "kalp", "tansiyon", "boyun", "kalori", "vitamin", "göğüs"]

siyasi_kavramlar1 = ["başkan", "dışişleri", "erdoğan", "avrupa", "abd", "pkk", "talabani", "almanya", "anayasa", "milletvekili", "müzakere" ]
siyasi_kavramlar2 = ["cumhurbaşkanı", "büyükşehir", "fransa", "chp", "akp", "tbmm", "yolsuzluk", "mahkeme" ]

spor_kavramlar1 = ["teknik", "futbol", "transfer", "fenerbahçe", "direktör", "futbolcu", "spor", "galatasaray", "sezon"]
spor_kavramlar2 = ["maç", "takım", "beşiktaş", "terim", "trabzonspor", "şampiyon", "lig", "pota"]

In [14]:
from snorkel.labeling import labeling_function
import re

@labeling_function()
def ekonomik_kavram_iceriyor1(sample_new):
    for ekonomik_kavram in ekonomi_kavramlar1:
         if re.search(ekonomik_kavram, sample_new.Haber):
            return EKONOMI
    return BELIRSIZ

@labeling_function()
def ekonomik_kavram_iceriyor2(sample_new):
    for sayi_birimi in ekonomi_kavramlar2:
        if re.search(sayi_birimi, sample_new.Haber):
            return EKONOMI 
    return BELIRSIZ

@labeling_function()
def magazin_kavrami_iceriyor1(sample_new):
    for magazin_kavrami in magazin_kavramlar1:
        if re.search(magazin_kavrami, sample_new.Haber):
            return MAGAZIN 
    return BELIRSIZ

@labeling_function()
def magazin_kavrami_iceriyor2(sample_new):
    for magazin_kisisi in magazin_kavramlar2:
        if re.search(magazin_kisisi, sample_new.Haber):
            return MAGAZIN 
    return BELIRSIZ

@labeling_function()
def saglik_kavramlari_iceriyor1(sample_new):
    for saglik_kavrami in saglik_kavramlar1:
        if re.search(saglik_kavrami, sample_new.Haber):
            return SAGLIK
    return BELIRSIZ

@labeling_function()
def saglik_kavramlari_iceriyor2(sample_new):
    for hastalik in saglik_kavramlar2:
        if re.search(hastalik, sample_new.Haber):
            return SAGLIK 
    return BELIRSIZ

@labeling_function()
def siyasi_kavram_iceriyor1(sample_new):
    for siyasi_kavram in siyasi_kavramlar1:
        if re.search(siyasi_kavram, sample_new.Haber):
            return SIYASI 
    return BELIRSIZ

@labeling_function()
def siyasi_kavram_iceriyor2(sample_new):
    for siyasi_parti in siyasi_kavramlar2:
        if re.search(siyasi_parti, sample_new.Haber):
            return SIYASI 
    return BELIRSIZ

@labeling_function()
def spor_kavramlar_iceriyor1(sample_new):
    for spor_kavrami in spor_kavramlar1:
        if re.search(spor_kavrami, sample_new.Haber):
            return SPOR 
    return BELIRSIZ

@labeling_function()
def spor_kavramlar_iceriyor2(sample_new):
    for takim in spor_kavramlar2:
        if re.search(takim, sample_new.Haber):
            return SPOR 
    return BELIRSIZ


In [15]:
from snorkel.labeling import PandasLFApplier

lfs = [ekonomik_kavram_iceriyor1, ekonomik_kavram_iceriyor2, magazin_kavrami_iceriyor1, magazin_kavrami_iceriyor2, saglik_kavramlari_iceriyor1, saglik_kavramlari_iceriyor2,
siyasi_kavram_iceriyor1, siyasi_kavram_iceriyor2, spor_kavramlar_iceriyor1, spor_kavramlar_iceriyor2]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_test = applier.apply(df=df_test)

100%|███████████████████████████████████████████████████████████████████████████████| 230/230 [00:00<00:00, 337.45it/s]


In [16]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
ekonomik_kavram_iceriyor1,0,[1],0.201087,0.196739,0.178261
ekonomik_kavram_iceriyor2,1,[1],0.408696,0.391304,0.372826
magazin_kavrami_iceriyor1,2,[2],0.342391,0.322826,0.311957
magazin_kavrami_iceriyor2,3,[2],0.078261,0.068478,0.057609
saglik_kavramlari_iceriyor1,4,[3],0.378261,0.359783,0.334783
saglik_kavramlari_iceriyor2,5,[3],0.217391,0.201087,0.176087
siyasi_kavram_iceriyor1,6,[4],0.517391,0.476087,0.436957
siyasi_kavram_iceriyor2,7,[4],0.195652,0.192391,0.153261
spor_kavramlar_iceriyor1,8,[5],0.253261,0.248913,0.227174
spor_kavramlar_iceriyor2,9,[5],0.271739,0.260870,0.239130


In [17]:
L_train.shape

(920, 10)

In [18]:
from snorkel.labeling.model import MajorityLabelVoter

majority_model = MajorityLabelVoter(6)
preds_train = majority_model.predict(L=L_train)

In [19]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=6, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)

In [20]:
Y_test = df_test.Sınıf.values

majority_acc = majority_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")
      
label_model_acc = label_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

Majority Vote Accuracy:   67.4%
Label Model Accuracy:     67.8%


In [21]:
from snorkel.labeling import filter_unlabeled_dataframe

probs_train = label_model.predict_proba(L=L_train)

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=df_train, y=probs_train, L=L_train
)

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1, 5))
X_train = vectorizer.fit_transform(df_train_filtered.Haber.tolist())
X_test = vectorizer.transform(df_test.Haber.tolist())

In [23]:
from snorkel.utils import probs_to_preds

preds_train_filtered = probs_to_preds(probs=probs_train_filtered)

### Logistic Regression

In [28]:
from sklearn.linear_model import LogisticRegression

sklearn_model = LogisticRegression(C=1e3, solver="liblinear")
sklearn_model.fit(X=X_train, y=preds_train_filtered)
print(f"Test Accuracy: {sklearn_model.score(X=X_test, y=Y_test) * 100:.1f}%")

D:\ProgramFiles\Anaconda3\envs\snorkel\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Test Accuracy: 81.3%


### Naive Bayes

In [26]:
from sklearn.metrics import accuracy_score
from sklearn import naive_bayes

# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, preds_train_filtered)

# predict the labels on validation dataset
predictions_NB = Naive.predict(X_test)

# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ", accuracy_score(predictions_NB, Y_test)*100)

Naive Bayes Accuracy Score ->  64.34782608695652


### SVM

In [27]:
from sklearn import svm
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, preds_train_filtered)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(X_test)

# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM, Y_test)*100)

SVM Accuracy Score ->  77.39130434782608
